In [1]:
import pandas as pd
import numpy as np
import nltk
import time
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob
import csv
from functools import reduce
import re
import string

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk import word_tokenize, pos_tag_sents, pos_tag
import re
from bs4 import BeautifulSoup
import unicodedata
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from string import punctuation
import os, glob
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
stops = set(stopwords.words("english")) 
wordnet_lemmatizer = WordNetLemmatizer()
from wordcloud import WordCloud
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Phoebe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Phoebe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Phoebe\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Phoebe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [26]:
labels = ["Direct Harassment", "Hate Speech","Sexual Harassment","Trolling", "Others", "Toxic"]
col = ["preprocessed_1", "Direct Harassment", "Hate Speech","Sexual Harassment","Trolling", "Others", "Toxic"]
folder = "../Annotations/CSVs/"

files = ['usanews.csv', 'foxnews0.csv', 'foxnews1.csv', 'foxnews2.csv', 'foxnews3.csv', 'nogla0.csv','pew0.csv', 'rae0_0.csv', 
         'rae0_1.csv', 'terror0_0.csv', 'terror0_1.csv']
data = ['preprocessed_1']

def combine_batch(files, name):
    dfs = []
    for file in files:
        df = pd.read_csv(folder+file)
        dfs.append(df)
    combined_df = pd.DataFrame(columns= dfs[0].columns )
    for df in dfs:
        combined_df= combined_df.append(df, ignore_index = True)
    combined_df.to_csv( "../Annotations/CSVs/"+name)
    return combined_df

In [27]:
foxnews = ['foxnews0.csv', 'foxnews1.csv', 'foxnews2.csv', 'foxnews3.csv']
rae = ['rae0_0.csv', 'rae0_1.csv']
terror = ['terror0_0.csv', 'terror0_1.csv']

fn = combine_batch(foxnews, "foxnews_FULL.csv")
r = combine_batch(rae, "rae_FULL.csv")
t = combine_batch(terror, "terror_FULL.csv")


In [28]:
def get_tfidf_vect(documents):
    vectorizer = TfidfVectorizer(ngram_range=(2,3), max_df=0.85, analyzer = 'word', min_df=1, max_features=10000, stop_words='english', lowercase=True)
    vector = vectorizer.fit_transform(documents)
    feature_names = np.array(vectorizer.get_feature_names())
    return pd.DataFrame(vector.todense(), columns=feature_names), vector, feature_names

In [30]:
def get_combined_tfidf_values_per_label(tfidf_df):
    combined_tfidf = pd.DataFrame(columns= (list(features) + labels))
    for label in labels:
        combined_scores = tfidf_df[tfidf_df[label] == 1][features].sum()
        to_append = list(combined_scores) + [0,0,0,0,0,0]
        row = pd.Series(to_append, index = (list(features) + labels))
        combined_tfidf = combined_tfidf.append(row, ignore_index=True)
        combined_tfidf.loc[combined_tfidf.index[-1],label] = 1.0
    return combined_tfidf

In [31]:
files_vid = ["foxnews_FULL.csv", "rae_FULL.csv", "terror_FULL.csv", "usanews.csv", "nogla0.csv", "pew0.csv"]
for file in files_vid:
    df = pd.read_csv(folder + file)
    df[labels] = df[labels].astype("float").replace(r'^\s*$', np.nan, regex=True).fillna(0).replace(np.nan,0)
    df[data] = df[data].replace(np.nan, '', regex=True)
    df["none"] = (df[labels].max(axis=1) == 0).astype(float)
    
    tfidf_df, vector, features = get_tfidf_vect(df['preprocessed_1'])
    tfidf_df[labels] = df[labels]
    combined_tfidf = get_combined_tfidf_values_per_label(tfidf_df)
    combined_tfidf.to_csv("tfidf_scores_results_per_video/td_" + file)